In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(1, '/gpfs/fs2/scratch/awhite38_lab/cfdsr/alcfd')
import utilities as utils

## Grid search

Defining input parameters

In [2]:
samples = 8

# grid
pipe_D = np.linspace(0.005, 0.1, samples)
bend_angle = np.linspace(1, 180, samples, dtype=int)
inlet_v = np.linspace(0.025, 0.005, samples)

inlet_p = np.array([101325 for _ in pipe_D]) # [Pa]
rho = np.array([998 for _ in pipe_D]) # [kg m^-3]
muo = np.array([9.737e-4 for _ in pipe_D]) # [Pa s]

#sort inputs by bend angle, helps run CFD faster
input_to_sort = np.array([[i,j,k] for i,j,k in zip(pipe_D, bend_angle, inlet_v)])
# input_to_sort[0] = [0.027, 165.0, 0.009]
input_to_sort = input_to_sort[input_to_sort[:, 1].argsort()]

pipe_D, bend_angle, inlet_v = input_to_sort[:,0], input_to_sort[:,1], input_to_sort[:,2]

inputs = [pipe_D, bend_angle, rho, muo, inlet_v, inlet_p]
print(inputs)

[array([0.005     , 0.01857143, 0.03214286, 0.04571429, 0.05928571,
       0.07285714, 0.08642857, 0.1       ]), array([  1.,  26.,  52.,  77., 103., 128., 154., 180.]), array([998, 998, 998, 998, 998, 998, 998, 998]), array([0.0009737, 0.0009737, 0.0009737, 0.0009737, 0.0009737, 0.0009737,
       0.0009737, 0.0009737]), array([0.025     , 0.02214286, 0.01928571, 0.01642857, 0.01357143,
       0.01071429, 0.00785714, 0.005     ]), array([101325, 101325, 101325, 101325, 101325, 101325, 101325, 101325])]


In [3]:
re_number = utils.check_laminar(pipe_D, rho, inlet_v, muo)
print(re_number)

[128.11954401 421.48715294 635.36835089 769.76313787 824.67151388
 800.0934789  696.02903295 512.47817603]


In [4]:
outputs = utils.run_model(*inputs, name='grid', debug=True)
outputs

No input was given for pipe L/D. Assuming L/D = 10.
Model inputs are saved in:
 /gpfs/fs2/scratch/hgandhi/cfdal/alcfd/examples/inputs_grid.txt
Results are stored in:
 /gpfs/fs2/scratch/hgandhi/cfdal/alcfd/examples/outputs_grid.txt.


,Pipe D,Elbow angle,Mesh size,Density,Viscosity,Inlet V,Inlet P,Pipe L/D,Del P/L
0,0.005000,1.0,0.000250,998.0,0.000974,0.025000,101325.0,10.0,31.188794
1,0.018571,26.0,0.000929,998.0,0.000974,0.022143,101325.0,10.0,2.531238
2,0.032143,52.0,0.001607,998.0,0.000974,0.019286,101325.0,10.0,1.077984
3,0.045714,77.0,0.002286,998.0,0.000974,0.016429,101325.0,10.0,0.566649
4,0.059286,103.0,0.002964,998.0,0.000974,0.013571,101325.0,10.0,0.284474
5,0.072857,128.0,0.003643,998.0,0.000974,0.010714,101325.0,10.0,0.150461
6,0.086429,154.0,0.004321,998.0,0.000974,0.007857,101325.0,10.0,0.079818
7,0.100000,180.0,0.005000,998.0,0.000974,0.005000,101325.0,10.0,0.036443


### Final model/descriptor to report - GRID
================================================================================

  4D descriptor (model):
Total RMSE,MaxAE:   0.000856  0.001932

@@@descriptor:

                     77:cos((angle)^-1)
                    100:(exp(pipe_D)/pipe_D)
                    220:((pipe_D)^-1/cos(angle))
                    240:(cos(pipe_D)-sin(inlet_V))
       coefficients_001:    -0.4122963469E+02    0.6755197113E-01   -0.4644300362E-04    0.4092794051E+02
          Intercept_001:     0.0000000000E+00
         RMSE,MaxAE_001:     0.8559426649E-03    0.1931984361E-02
         

$$-41.22 \cos\big(\frac{1}{\theta}\big) + 0.06755 \frac{\exp(D)}{D} - 0.000046 \frac{1}{D \cos(\theta)} + 40.927 \big(\cos(D) - \sin(v)\big)$$

## 3D grid search

In [7]:
# 3D grid
dim = 10
D = np.linspace(0.005, 0.1, dim)
theta = np.linspace(1, 180, dim, dtype=int)
v = np.linspace(0.02, 0.005, dim)
dd, tt, vv = np.meshgrid(D, theta, v, indexing='ij')
grid_3d = np.array([(i, j, k) for i,j,k in zip(dd.flatten(), tt.flatten(), vv.flatten())])
samples = grid_3d.shape[0]
samples

1000

In [8]:
inlet_p = np.array([101325 for _ in range(samples)]) # [Pa]
rho = np.array([998 for _ in range(samples)]) # [kg m^-3]
muo = np.array([9.737e-4 for _ in range(samples)]) # [Pa s]

inputs_3d_grid = [grid_3d[:,0], grid_3d[:,1], rho, muo, grid_3d[:,2], inlet_p]
grid_3d[4], grid_3d[6] #, grid_3d[24]

(array([0.005     , 1.        , 0.01333333]), array([0.005, 1.   , 0.01 ]))

In [9]:
re_number = utils.check_laminar(grid_3d[:,0], rho, grid_3d[:,2], muo)
print(re_number)

[ 102.49563521   93.95433227   85.41302934   76.8717264    68.33042347
   59.78912054   51.2478176    42.70651467   34.16521174   25.6239088
  102.49563521   93.95433227   85.41302934   76.8717264    68.33042347
   59.78912054   51.2478176    42.70651467   34.16521174   25.6239088
  102.49563521   93.95433227   85.41302934   76.8717264    68.33042347
   59.78912054   51.2478176    42.70651467   34.16521174   25.6239088
  102.49563521   93.95433227   85.41302934   76.8717264    68.33042347
   59.78912054   51.2478176    42.70651467   34.16521174   25.6239088
  102.49563521   93.95433227   85.41302934   76.8717264    68.33042347
   59.78912054   51.2478176    42.70651467   34.16521174   25.6239088
  102.49563521   93.95433227   85.41302934   76.8717264    68.33042347
   59.78912054   51.2478176    42.70651467   34.16521174   25.6239088
  102.49563521   93.95433227   85.41302934   76.8717264    68.33042347
   59.78912054   51.2478176    42.70651467   34.16521174   25.6239088
  102.4956352

In [ ]:
outputs_3d_grid = utils.run_model(*inputs_3d_grid, name='grid_100_3d', debug=True)
outputs_3d_grid

No input was given for pipe L/D. Assuming L/D = 10.
Model inputs are saved in:
 /gpfs/fs2/scratch/hgandhi/cfdal/alcfd/examples/inputs_grid_100_3d.txt


### Final model/descriptor to report - 3D grid
================================================================================

  4D descriptor (model):  
Total RMSE,MaxAE:   0.000843  0.001534

@@@descriptor:

                     28:(sin(Inlet_V)/(angle*Inlet_V))
                     52:(cos(Pipe_D)/(angle*Inlet_V))                     
                     65:(cos(Pipe_D)/(angle*Pipe_D))                     
                    128:(sin(Pipe_D)/(angle*Inlet_V))                    
       coefficients_001:     0.4463515868E+05   -0.1159693877E+04   -0.3839990797E+00    0.9323306506E+04
          Intercept_001:     0.0000000000E+00          
         RMSE,MaxAE_001:     0.8429907482E-03    0.1533815868E-02

## Random search

In [13]:
# random
pipe_D = np.round(np.random.choice(np.linspace(0.005, 0.1, 1000), size=samples), decimals=3) # [m]
bend_angle = np.random.choice(np.arange(1, 181), size=samples) # [degrees]
inlet_v = np.round(np.random.choice(np.linspace(0.025, 0.005, 1000), size=samples), decimals=3) # [m/s]

#sort inputs by bend angle, helps run CFD faster
input_to_sort = np.array([[i,j,k] for i,j,k in zip(pipe_D, bend_angle, inlet_v)])
# input_to_sort[0] = [0.027, 165.0, 0.009]
input_to_sort = input_to_sort[input_to_sort[:, 1].argsort()]
pipe_D, bend_angle, inlet_v = input_to_sort[:,0], input_to_sort[:,1], input_to_sort[:,2]
inputs = [pipe_D, bend_angle, rho, muo, inlet_v, inlet_p]
print(inputs)

[array([0.009, 0.024, 0.014, 0.065, 0.033, 0.092, 0.008, 0.053]), array([ 27.,  46.,  70.,  87., 115., 137., 158., 175.]), array([998, 998, 998, 998, 998, 998, 998, 998]), array([0.0009737, 0.0009737, 0.0009737, 0.0009737, 0.0009737, 0.0009737,
       0.0009737, 0.0009737]), array([0.018, 0.01 , 0.019, 0.02 , 0.015, 0.014, 0.022, 0.005]), array([101325, 101325, 101325, 101325, 101325, 101325, 101325, 101325])]


In [14]:
re_number = utils.check_laminar(pipe_D, rho, inlet_v, muo)
print(re_number)

[ 166.04292903  245.98952449  272.63838965 1332.44325768  507.35339427
 1320.14378145  180.39231796  271.6134333 ]


In [15]:
outputs = utils.run_model(*inputs, name='random_8', debug=True)
outputs

No input was given for pipe L/D. Assuming L/D = 10.
Model inputs are saved in:
 /gpfs/fs2/scratch/hgandhi/cfdal/alcfd/examples/inputs_random_8.txt
Results are stored in:
 /gpfs/fs2/scratch/hgandhi/cfdal/alcfd/examples/outputs_random_8.txt.


,Pipe D,Elbow angle,Mesh size,Density,Viscosity,Inlet V,Inlet P,Pipe L/D,Del P/L
0,0.009,27.0,0.00045,998.0,0.000974,0.018,101325.0,10.0,7.906819
1,0.024,46.0,0.00120,998.0,0.000974,0.010,101325.0,10.0,0.745303
2,0.014,70.0,0.00070,998.0,0.000974,0.019,101325.0,10.0,4.845758
3,0.065,87.0,0.00325,998.0,0.000974,0.020,101325.0,10.0,0.430883
4,0.033,115.0,0.00165,998.0,0.000974,0.015,101325.0,10.0,0.870834
5,0.092,137.0,0.00460,998.0,0.000974,0.014,101325.0,10.0,0.153772
6,0.008,158.0,0.00040,998.0,0.000974,0.022,101325.0,10.0,18.062219
7,0.053,175.0,0.00265,998.0,0.000974,0.005,101325.0,10.0,0.105963


In [17]:
print(outputs.drop(columns=['Mesh size', 'Density', 'Viscosity', 'Inlet P', 'Pipe L/D']))

   Pipe D  Elbow angle  Inlet V    Del P/L
0   0.009         27.0    0.018   7.906819
1   0.024         46.0    0.010   0.745303
2   0.014         70.0    0.019   4.845758
3   0.065         87.0    0.020   0.430883
4   0.033        115.0    0.015   0.870834
5   0.092        137.0    0.014   0.153772
6   0.008        158.0    0.022  18.062219
7   0.053        175.0    0.005   0.105963


### Final model/descriptor to report - Random
================================================================================

  4D descriptor (model):
Total RMSE,MaxAE:   0.005820  0.011148
@@@descriptor:

                      6:[((Inlet_v/Pipe_D)/Pipe_D)]
                    124:[((Inlet_v/Pipe_D)/(angle/Inlet_v))]
                    235:[(cos(Inlet_v)/(Pipe_D/Inlet_v))]
                    236:[((Pipe_D)^-1*sin(Inlet_v))]
       coefficients_001:     0.4249892889E-01   -0.3051365588E+04   -0.7783333345E+04    0.7783758486E+04
          Intercept_001:     0.0000000000E+00
         RMSE,MaxAE_001:     0.5820341983E-02    0.1114776354E-01
         


## AL - 8 points

### Model/descriptor for generating residual:
================================================================================

  3D descriptor (model):
Total RMSE,MaxAE:   0.025450  0.057487
@@@descriptor:

                      2:[((Inlet_v/Pipe_D)/sin(Pipe_D))]
                    110:[(exp(Inlet_v)/(angle*Inlet_v))]
                    277:[(exp(Pipe_D)/(Pipe_D/Inlet_v))]
       coefficients_001:     0.3522745829E-01   -0.9372233079E-01    0.9654343163E+00
          Intercept_001:     0.0000000000E+00
         RMSE,MaxAE_001:     0.2545010415E-01    0.5748714861E-01

$$ 0.035 \frac{v}{D \sin(D)} -0.093 \frac{\exp(v)}{\theta v} + 0.965 \frac{\exp(D) v}{D} $$ 